<a href="https://colab.research.google.com/github/jaschn/DeepLearningProject/blob/main/Comparisons/InstColor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colorizing images with instance aware image colorization model

### Cloning the papers guys repo

In [1]:
import os

!git clone https://github.com/ericsujw/InstColorization

Cloning into 'InstColorization'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 211 (delta 15), reused 11 (delta 11), pack-reused 177
Receiving objects: 100% (211/211), 6.11 MiB | 14.58 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [2]:
os.chdir('InstColorization')
!ls

checkpoints  fusion_dataset.py	     LICENSE	      scripts
data	     image_util.py	     models	      test_fusion.py
download.py  imgs		     options	      train.py
env.yml      inference_bbox.py	     README.md	      util
example      InstColorization.ipynb  README_TRAIN.md


### Getting the real image data 

In [3]:
import numpy as np
import glob

from google.colab import drive
drive.mount('/content/drive')

drive_path_real = '/content/drive/MyDrive/DeepLearnProject/posterImgsReal'
paths = glob.glob(drive_path_real + "/*.jpg") # Grabbing all the image file names

#!pip install fastai --upgrade
#from fastai.data.external import untar_data, URLs

#coco_path = untar_data(URLs.COCO_SAMPLE)
#coco_path = str(coco_path) + "/train_sample"

#paths = glob.glob(coco_path + "/*.jpg") # Grabbing all the image file names
#np.random.seed(555)
#paths_subset = np.random.choice(paths, 10, replace=False) # choosing 100 images random

Mounted at /content/drive


### Getting our fake image data

In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

drive_path_cartoon = '/content/drive/MyDrive/DeepLearnProject/posterImgsCartoon'

image_paths_cartoon_jpg = glob.glob(drive_path_cartoon + "/*.jpg")
image_paths_cartoon_png = glob.glob(drive_path_cartoon + "/*.png")

image_paths_cartoon = image_paths_cartoon_jpg + image_paths_cartoon_png

### Putting our images in the `example` folder

In [5]:
#Put our images in the example folder
import shutil

for file in paths:
  shutil.copy(file, 'example/')

for file in image_paths_cartoon:
  shutil.copy(file, 'example/')

### Install other dependencies

In [6]:
! sh scripts/install.sh

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8 MB 22 kB/s 
     |████████████████████████████████| 6.6 MB 47.6 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.5.0+cu101 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.5.0+cu101 which is incompatible.
     |████████████████████████████████| 274 kB 4.3 MB/s 
  Created wheel fo

### Downloading the pretrained model into checkpoints

In [7]:
!sh scripts/download_model.sh

Downloading...
Finish download.
Archive:  checkpoints.zip
   creating: checkpoints/coco_finetuned_mask_256_ffs/
  inflating: checkpoints/coco_finetuned_mask_256_ffs/latest_net_GComp.pth  
  inflating: checkpoints/coco_finetuned_mask_256_ffs/latest_net_G.pth  
  inflating: checkpoints/coco_finetuned_mask_256_ffs/latest_net_GF.pth  
   creating: checkpoints/coco_finetuned_mask_256/
  inflating: checkpoints/coco_finetuned_mask_256/latest_net_GComp.pth  
  inflating: checkpoints/coco_finetuned_mask_256/latest_net_G.pth  
  inflating: checkpoints/coco_finetuned_mask_256/latest_net_GF.pth  
   creating: checkpoints/siggraph_retrained/
  inflating: checkpoints/siggraph_retrained/latest_net_G.pth  


In [10]:
#Getting OUR trained model checkpoints from drive and substituting them into the checkpoint folder instead of the pretrained ones

drive_path_latestnetG = '/content/drive/MyDrive/coco_mask_cartoon/latest_net_G.pth'
drive_path_latestnetGComp = '/content/drive/MyDrive/coco_mask_cartoon/latest_net_GComp.pth'
drive_path_latestnetGF = '/content/drive/MyDrive/coco_mask_cartoon/latest_net_GF.pth'

checkpointPaths = [drive_path_latestnetG, drive_path_latestnetGComp, drive_path_latestnetGF]

for file in checkpointPaths:
  shutil.copy(file, 'checkpoints/coco_finetuned_mask_256_ffs/')

### Finding the bboxes of the images in the example folder

In [11]:
!python inference_bbox.py --test_img_dir example

model_final_2d9806.pkl: 431MB [00:36, 11.8MB/s]               
Create path: example_bbox
100% 18/18 [00:14<00:00,  1.26it/s]


### Colorizing the images in the `example` folder

In [12]:
!python test_fusion.py --name test_fusion --sample_p 1.0 --model fusion --fineSize 256 --test_img_dir example --results_img_dir results

Create path: results
#Testing images = 18
initialize network with normal
initialize network with normal
initialize network with normal
model [FusionModel] was created
load Fusion model from checkpoints/coco_finetuned_mask_256_ffs/latest_net_GF.pth
  0% 0/18 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
100% 18/18 [00:43<00:00,  2.44s/it]
2 images without bounding boxes


### Downloading the results

In [13]:
!zip -r /content/resultfile.zip /content/InstColorization/results
!zip -r /content/examplefile.zip /content/InstColorization/example

from google.colab import files
files.download("/content/resultfile.zip")
files.download("/content/examplefile.zip")

  adding: content/InstColorization/results/ (stored 0%)
  adding: content/InstColorization/results/000000575867.png (deflated 0%)
  adding: content/InstColorization/results/000000496499.png (deflated 0%)
  adding: content/InstColorization/results/138.png (deflated 0%)
  adding: content/InstColorization/results/000000494391.png (deflated 1%)
  adding: content/InstColorization/results/144.png (deflated 0%)
  adding: content/InstColorization/results/574100c6e31a1dcd096476eb2d632b3c.png (deflated 1%)
  adding: content/InstColorization/results/000000023781.png (deflated 1%)
  adding: content/InstColorization/results/000000133933.png (deflated 0%)
  adding: content/InstColorization/results/19.png (deflated 0%)
  adding: content/InstColorization/results/e9b6826aa623549ec77bbc0275002779.png (deflated 1%)
  adding: content/InstColorization/results/000000050145.png (deflated 1%)
  adding: content/InstColorization/results/8b01a894fb91025a1dc77611838e9d13.png (deflated 0%)
  adding: content/InstCo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

FileNotFoundError: ignored

### Result visualization